In [1]:
from utils import normalize_df
import os
notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
data_dir = os.path.join(root_dir, 'data')
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss, make_scorer, log_loss, mean_squared_error
from IPython.display import display_html
from copy import deepcopy
import pickle
from sklearn import calibration
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GroupKFold, RandomizedSearchCV, cross_val_predict, GridSearchCV
# from utils.utils

In [2]:
def create_train_test_val_df(
    df,
    input_names,
    output_name,
    group_col="game_code",
    mask_test_season=2021,
    mask_val_season=[2019, 2020],
    normalize=False
):
    mask_train = ~(df.season.isin([mask_test_season] + mask_val_season))
    mask_test = (df.season == mask_test_season)
    mask_val = (df.season.isin(mask_val_season))
    if normalize==False:
        X_train = df.loc[mask_train, input_names]
        X_test = df.loc[mask_test, input_names]
        X_val = df.loc[mask_val, input_names]
    else:
        X_train = normalize_df(df.loc[mask_train, input_names])
        X_test = normalize_df(df.loc[mask_test, input_names], df.loc[mask_train, input_names])
        X_val = normalize_df(df.loc[mask_val, input_names], df.loc[mask_train, input_names])
    y_train = df[mask_train][output_name]
    group_train = df[mask_train][group_col]
    y_test = df[mask_test][output_name]
    group_test = df[mask_test][group_col]
    y_val = df[mask_val][output_name]
    group_val = df[mask_val][group_col]
    return X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val

# Model 1.0
This notebook will act as an interactive tutorial for our Live Win Probability Model. This "model" is actual comprised of 3 separate models that "stack" on each other.
1. Play and drive outcome models
    * technically this is two separate models:
        * Play outcome (first down, field goal made, field goal missed, touchdown, turnover, and none/other)
            * only using the first down prediction from the output of this model
        * Drive outcome (Clock, field goal made, field goal missed, punt, safety, touch down, turnover, turnover on downs)
    * outputs for both models will be a series of probabilities for each class that all add up to 1
2. End of regulation score differential model
    * Dealing with overtime later, we want to predict how the score differential will change by the end of regulation.
        * i.e., if the current score differential (home score - away score) is -3 and the end of regulation score differential is -10, the target value will be -7
    * Output of this will be a series of probabilities from for all score differential possibilities from -35 to 35 (outputs <-35 or >35 will be set to -35/35 respectively)
3. End of regulation score total model
    * Similar concept to the score differential model
    * Again, we're using the change in end of regulation score total as the target value
    * Outputs will be a series of probabilites for classes from 0 to 83 (outputs will be capped at 83)

## Data
Let's take a look at the data that we are pulling from oracle
* First we have event_df and odds_df
* event_df is the play by play data mixed with some import game information
* Odds data has vegas predictions for almost all the games in the set (missing games will be given the average vegas spread and over/under)
    * The spread and over/under are merged with the event table to give us our pre-game priors
    * some games have multiple odds so duplicates are removed


In [3]:
event_df = pd.read_parquet(os.path.join(data_dir, "event_data_cfb.parquet"))
event_df = event_df.drop_duplicates(["nevent", "game_code"]).reset_index(drop=True)
odds_df = pd.read_parquet(os.path.join(data_dir, "odds_data_cfb.parquet"))
odds_df = odds_df.drop_duplicates("game_code")
event_df[["cur_spread", "cur_over_under"]] = event_df.merge(odds_df, how="left", on="game_code")[["cur_spread", "cur_over_under"]].fillna({"cur_spread": np.mean(odds_df["cur_spread"]), "cur_over_under": np.mean(odds_df["cur_over_under"])})
pd.set_option("display.max_columns", None)
display_html(event_df)

,game_code,game_date,season,home_team_id,home_team,home_team_abbrev,home_conf_name,home_division_type,away_team_id,away_team,away_team_abbrev,away_conf_name,away_division_type,home_final_score,away_final_score,final_score_diff,end_of_regulation_score_diff,home_rest_of_game_score,away_rest_of_game_score,end_of_regulation_score_diff_change,home_score_added,away_score_added,current_score_diff,current_score_total,home_start_score,away_start_score,home_team_outcome,home_team_win,draw,away_team_win,nevent,quarter,overtime,home_team_has_ball,off_team_id,def_team_id,kick_off,punt,point_after_kick,two_point_attempt,field_goal_attempt,off_start_score,off_end_score,off_score_change,def_start_score,def_end_score,def_score_change,play_counts,efficiency_counts,from_scrimmage,first_down,scoring_play,possession_change,continuation,event_name,event_id,yards_gained,drive_outcome_id,drive_outcome_desc,down,ytg,yd_from_goal,drive_id,drive_start,play_start_time,cur_spread,cur_over_under
0,1178084,2012-08-30 17:15:00,2012,3499,Utah Utes,Utah,Pac-12,FBS,3710,Northern Colorado Bears,NorCol,Big Sky,FCS,41,0,41,41,41,0,41,0,0,0,0,0,0,W,1,0,0,1,1,0,0,3710,3499,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,Kick Off,5,65.0,NaN,None,0,-1,65,NaN,3600.0,900.0,-3.308451,55.707539
1,1178084,2012-08-30 17:15:00,2012,3499,Utah Utes,Utah,Pac-12,FBS,3710,Northern Colorado Bears,NorCol,Big Sky,FCS,41,0,41,41,41,0,41,0,0,0,0,0,0,W,1,0,0,2,1,0,1,3499,3710,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,Touchback,21,0.0,NaN,None,0,-1,100,NaN,3600.0,NaN,-3.308451,55.707539
2,1178084,2012-08-30 17:15:00,2012,3499,Utah Utes,Utah,Pac-12,FBS,3710,Northern Colorado Bears,NorCol,Big Sky,FCS,41,0,41,41,41,0,41,0,0,0,0,0,0,W,1,0,0,3,1,0,1,3499,3710,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,Run,4,5.0,9.0,Intercepted Pass,1,10,75,1.0,3600.0,900.0,-3.308451,55.707539
3,1178084,2012-08-30 17:15:00,2012,3499,Utah Utes,Utah,Pac-12,FBS,3710,Northern Colorado Bears,NorCol,Big Sky,FCS,41,0,41,41,41,0,41,0,0,0,0,0,0,W,1,0,0,4,1,0,1,3499,3710,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,Run,4,4.0,9.0,Intercepted Pass,2,5,70,1.0,3600.0,866.0,-3.308451,55.707539
4,1178084,2012-08-30 17:15:00,2012,3499,Utah Utes,Utah,Pac-12,FBS,3710,Northern Colorado Bears,NorCol,Big Sky,FCS,41,0,41,41,41,0,41,0,0,0,0,0,0,W,1,0,0,5,1,0,1,3499,3710,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,Run,4,11.0,9.0,Intercepted Pass,3,1,66,1.0,3600.0,839.0,-3.308451,55.707539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930683,2405840,2022-01-10 20:00:00,2021,3478,Alabama Crimson Tide,Ala,Southeastern,FBS,3473,Georgia Bulldogs,UGa,Southeastern,FBS,18,33,-15,-15,0,0,0,0,0,-15,51,18,33,L,0,0,1,234,4,0,1,3478,3473,0,0,0,0,0,18,18,0,33,33,0,1,1,1,0,0,0,0,Sack,3,-6.0,40.0,Downs,2,22,58,13.0,54.0,34.0,3.000000,53.000000
1930684,2405840,2022-01-10 20:00:00,2021,3478,Alabama Crimson Tide,Ala,Southeastern,FBS,3473,Georgia Bulldogs,UGa,Southeastern,FBS,18,33,-15,-15,0,0,0,0,0,-15,51,18,33,L,0,0,1,235,4,0,1,3478,3473,0,0,0,0,0,18,18,0,33,33,0,1,0,0,0,0,0,0,Offense Timeout,57,NaN,40.0,Downs,3,28,64,13.0,54.0,NaN,3.000000,53.000000
1930685,2405840,2022-01-10 20:00:00,2021,3478,Alabama Crimson Tide,Ala,Southeastern,FBS,3473,Georgia Bulldogs,UGa,Southeastern,FBS,18,33,-15,-15,0,0,0,0,0,-15,51,18,33,L,0,0,1,236,4,0,1,3478,3473,0,0,0,0,0,18,18,0,33,33,0,1,1,1,0,0,0,0,Pass Completion,1,17.0,40.0,Downs,3,28,64,13.0,54.0,28.0,3.000000,53.000000
1930686,2405840,2022-01-10 20:00:00,2021,3478,Alabama Crimson Tide,Ala,Southeastern,FBS,3473,Georgia Bulldogs,UGa,Southeastern,FBS,18,33,-15,-15,0,0,0,0,0,-15,51,18,33,L,0,0,1,237,4,0,1,3478,3473,0,0,0,0,0,18,18,0,33,33,0,1,1,1,0,0,1,0,Sack,3,-12.0,40.0,Downs,4,11,47,13.0,54.0,4.0,3.000000,53.000000


Adding timeouts remaining for both teams and time left in game

In [4]:
event_df["half"] = round((event_df["quarter"] + 0.01) / 2)
event_df["home_timeout"] = np.where(((event_df["event_id"]==57)&(event_df["home_team_has_ball"]==1))|((event_df["event_id"]==58)&(event_df["home_team_has_ball"]==0)), 1, 0)
event_df["away_timeout"] = np.where(((event_df["event_id"]==57)&(event_df["home_team_has_ball"]==0))|((event_df["event_id"]==58)&(event_df["home_team_has_ball"]==1)), 1, 0)
event_df["home_timeouts_remaining"] = np.clip(3 - event_df.groupby(["game_code", "half"])["home_timeout"].cumsum(), 0, 3)
event_df["away_timeouts_remaining"] = np.clip(3 - event_df.groupby(["game_code", "half"])["away_timeout"].cumsum(), 0, 3)
event_df["time_left_in_game"] = np.where(event_df["quarter"] <= 4, event_df["play_start_time"] + (4 - event_df["quarter"]) * 900, event_df["play_start_time"])
event_df["time_left_in_game"] = event_df["time_left_in_game"].fillna(event_df["time_left_in_game"].shift(1))
# event_df["time_elapsed"] = 900 - event_df["play_start_time"] + (event_df["quarter"] - 1) * 900


* Our PBP will have multiple rows for one play, so if there's a fumble then recovery by offense and a touchdown, 
* that could have 2-3 rows of data and the touchdown wouldn't show up as being apart of the original play 
    * plays would look like this: 1. Run, 2. Fumble, 3. Offense Recovers the ball (TD)
* So what we've done here is ensure that plays that are "continuation" that end in a touchdown, give a TD=True for all of the plays in the sequence
* After that is taken care of we can setup all of the labels for play and drive description

In [5]:
event_df["sequence"] = event_df["continuation"].groupby(event_df["continuation"].eq(0).cumsum()).cumsum()
event_df["play_start_id"] = event_df["nevent"] - event_df["sequence"]

turnover_ids = [9, 16]
event_df["turnover"] = np.where(event_df["event_id"].isin(turnover_ids), 1, 0)
event_df["touchdown_scored"] = np.where(event_df["home_score_added"]+event_df["away_score_added"]>=6, 1, 0)
event_df["fieldgoal_made"] = np.where(event_df["home_score_added"]+event_df["away_score_added"]==3, 1, 0)

play_outcome_aggregate =event_df[["game_code", "play_start_id", "turnover", "touchdown_scored", "fieldgoal_made", "first_down"]].groupby(["game_code", "play_start_id"], as_index=False).sum()
event_df["touchdown_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["touchdown_scored_y"], 0, 1)
event_df["turnover_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["turnover_y"], 0, 1)
event_df["field_goal_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["fieldgoal_made_y"], 0, 1)
event_df["first_down_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["first_down_y"], 0, 1)



event_df["play_outcome"] = (
    np.where((event_df["turnover_in_play"]==1), "turnover",
    # (event_df["touchdown_in_play"]==1)&(event_df["turnover_in_play"]==1), "defensive_touchdown", 
    np.where((event_df["punt"]==1), "punt",
    np.where((event_df["field_goal_in_play"]==1), "field_goal_made",
    np.where((event_df["field_goal_attempt"]==1)&(event_df["field_goal_in_play"]==0), "field_goal_missed",
    np.where((event_df["first_down_in_play"]==1)&(event_df["touchdown_in_play"]==0)&(event_df["turnover_in_play"]==0)&(event_df["punt"]==0), "first_down",
    np.where((event_df["touchdown_in_play"]==1)&(event_df["turnover_in_play"]==0), "offensive_touchdown", "none"))))))
)
drive_description_matrix = {
    7: "punt",
    9: "turnover",
    14: "turnover",
    17: "field_goal_made",
    18: "punt",
    20: "safety",
    35: "field_goal_missed",
    36: "field_goal_missed",
    37: "touch_down",
    38: "clock",
    39: "clock",
    40: "turnover_on_downs",
    42: "field_goal_made",
    51: "clock",
}
event_df["drive_outcome_desc_basic"] = event_df["drive_outcome_id"].map(drive_description_matrix)

# event_df["drive_outcome"] = np.where(
#     (event_df["touchdown_in_drive"]==1)&(event_df["turnover_in_drive"]==1), "defensive_touchdown", 
#     np.where((event_df["touchdown_in_drive"]==1)&(event_df["turnover_in_drive"]==0), "offensive_touchdown",
#     np.where((event_df["field_goal_in_drive"]==1), "field_goal_made",
#     np.where((event_df["touchdown_in_drive"]==0)&(event_df["turnover_in_drive"]==1), "turnover", "none"
# ))))
game_end_of_regulation_total_score = event_df[event_df.overtime==0].groupby("game_code", as_index=False).max()[["game_code", "home_start_score", "away_start_score"]]
game_end_of_regulation_total_score["end_of_regulation_score_total"] = game_end_of_regulation_total_score["home_start_score"] + game_end_of_regulation_total_score["away_start_score"]
# event_df["end_of_regulation_score_total_diff"] = 
event_df["end_of_regulation_score_total_diff"] = (
    event_df.merge(game_end_of_regulation_total_score, on="game_code")["end_of_regulation_score_total"]
    - (event_df["home_start_score"] + event_df["away_start_score"])
)

/var/folders/dq/tcxv3s4j7y585_4jhfbdvrx00000gn/T/ipykernel_13817/1115593539.py:50: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  game_end_of_regulation_total_score = event_df[event_df.overtime==0].groupby("game_code", as_index=False).max()[["game_code", "home_start_score", "away_start_score"]]


### Data Manipulation
* We need to do a little bit of data manipulation to get the values we need, but we don't want to "overwrite" the values in event_df so we'll make copy of it called model_df
* time left in half is added
* from_scrimmage is changed so that PATs and two point conversions are not included
* down, ytg, and yd_from_goal are changed so that all non-scrimmage plays are changed to a default "null" value
* home_team_has_ball is change so that when kickoffs occur, the team receiving is the one that is in possession of the ball

### Data Subset
* Removing continuation plays that we mentioned before, so that each snap has just one target
* Remove plays where the down is equal to 0 
* Remove plays from scrimmage that did not count (e.g., plays that were waved off by penalties)
* scrimmage_plays_we_want is event_id of all the scrimmage plays that *aren't* timeouts, end of quarters, and the two minute warning.
* Remove all NA values for the feature inputs and target
* Remove all plays that are not from scrimmage
* Remove all overtime plays

In [6]:
model_df = deepcopy(event_df)
model_df["time_left_in_half"] = event_df["time_left_in_game"] - ((2 - event_df["half"]) * 1800)
model_df["from_scrimmage"] = np.where(event_df["event_id"].isin([22, 47, 52, 53, 54, 55, 56]), 0, event_df["from_scrimmage"])
model_df["down"] = np.where(model_df["from_scrimmage"] == 0, 0, event_df["down"])
model_df["ytg"] = np.where(model_df["from_scrimmage"] == 0, -1, event_df["ytg"])
model_df["yd_from_goal"] = np.where(model_df["from_scrimmage"] == 0, -1, event_df["yd_from_goal"])
model_df["home_team_has_ball"] = np.where(event_df["event_id"].isin([5]), 1 - event_df["home_team_has_ball"], event_df["home_team_has_ball"])
scrimmage_plays_we_want = [1, 2, 3, 4, 7, 9, 14, 17, 18, 35]


In [7]:

input_names = [
    'time_left_in_half',
    'half',
    'current_score_diff',
    'current_score_total',
    'cur_spread',
    'cur_over_under',
    'home_timeouts_remaining',
    'away_timeouts_remaining',
    # 'punt',
    # 'field_goal_attempt',
    'ytg',
    'yd_from_goal',
    'down',
    'home_team_has_ball',
]
output_name = "play_outcome"
mask_model = (
    (model_df.continuation==0)&
    (model_df.down!=0)&
    (model_df.play_counts==1)&
    (model_df.event_id.isin(scrimmage_plays_we_want))&
    (model_df[input_names+[output_name]].notna().all(axis=1))&
    (model_df["from_scrimmage"]==1)&
    (model_df["overtime"]==0)
)
X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val = create_train_test_val_df(model_df[mask_model], input_names, output_name)


Let's take a look at what the input features and output features look like. 

Event Name and yards gained is included to help interpret what is going on. 

This is the first 2 drives of the first game in the dataset

In [ ]:
model_df[mask_model][input_names + ["event_name", "yards_gained", "play_outcome", "drive_outcome_desc_basic"]].head(15)

### Loading in the Models and Creating Play/Drive Predictions
* ~~For this exercise we won't be training the models, just loading saved models and then using them to make predictions~~
* Now we are going to train the models
* In addition, each prediction will be split up between home and away. So if the home team has the ball the predictions for the away team play/drive outcomes are going to be set to 0


Let's take a look at how the predictions look for our dataset

In [8]:
cv=GroupKFold(n_splits=3)

rf_grid = {
    "n_estimators": np.linspace(start=50, stop=500, num=10, dtype=int),
    "max_features": ["auto", "sqrt"],
    "max_depth": np.linspace(5, 15, num=11, dtype=int),
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False],
}

log_loss_scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
rf_play_outcome_search_model = RandomForestClassifier(verbose=100, n_jobs=-1, random_state=1)
search_rf_play_outcome = RandomizedSearchCV(rf_play_outcome_search_model, rf_grid, cv=cv,random_state=42,n_iter=10,n_jobs=1,verbose=100, scoring=log_loss_scorer)
# rf_play_outcome_search_model = RandomForestClassifier(verbose=100, n_jobs=1, random_state=1)
# search_rf_play_outcome = RandomizedSearchCV(rf_play_outcome_search_model, rf_grid, cv=cv,random_state=42,n_iter=10,n_jobs=-1,verbose=100, scoring=log_loss_scorer)
search_rf_play_outcome.fit(X_train,y_train,groups=group_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START bootstrap=False, max_depth=1, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1
[CV 2/3; 1/10] START bootstrap=False, max_depth=1, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1
[CV 3/3; 1/10] START bootstrap=False, max_depth=1, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1
[CV 1/3; 2/10] START bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1
[CV 2/3; 2/10] START bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1
[CV 3/3; 2/10] START bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1
[CV 1/3; 3/10] START bootstrap=True, max_depth=1, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1
[CV 2/3; 3/10] START bootstrap=True, max_depth=1, m

RandomizedSearchCV(cv=GroupKFold(n_splits=3),
                   estimator=RandomForestClassifier(n_jobs=1, random_state=1,
                                                    verbose=100),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])},
                   random_state=42,
                   scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
                   verbose=100)

In [9]:
output_name = "drive_outcome_desc_basic"
mask_model = (
    (model_df.continuation==0)&
    (model_df.down!=0)&
    (model_df.play_counts==1)&
    (model_df.event_id.isin(scrimmage_plays_we_want))&
    (model_df[input_names+[output_name]].notna().all(axis=1))&
    (model_df["from_scrimmage"]==1)&
    (model_df["overtime"]==0)
)

X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val = create_train_test_val_df(model_df[mask_model], input_names, output_name)
cv=GroupKFold(n_splits=3)

log_loss_scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
rf_drive_outcome_search_model = RandomForestClassifier(verbose=100, n_jobs=-1, random_state=1)
search_rf_drive_outcome = RandomizedSearchCV(rf_drive_outcome_search_model, rf_grid, cv=cv,random_state=42,n_iter=10,n_jobs=1,verbose=100, scoring=log_loss_scorer)
search_rf_drive_outcome.fit(X_train,y_train,groups=group_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START bootstrap=False, max_depth=1, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1
[CV 2/3; 1/10] START bootstrap=False, max_depth=1, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1
[CV 3/3; 1/10] START bootstrap=False, max_depth=1, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1
[CV 1/3; 2/10] START bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1
[CV 2/3; 2/10] START bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1
[CV 3/3; 2/10] START bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1
[CV 1/3; 3/10] START bootstrap=True, max_depth=1, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1
[CV 2/3; 3/10] START bootstrap=True, max_depth=1, m

RandomizedSearchCV(cv=GroupKFold(n_splits=3),
                   estimator=RandomForestClassifier(n_jobs=1, random_state=1,
                                                    verbose=100),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])},
                   random_state=42,
                   scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
                   verbose=100)

## Storing Predictions
* We'll add in the plays that didn't count
* The predictions are broken up by home/away team

In [10]:
mask_model_predict = (
    (model_df.continuation==0)&
    (model_df.down!=0)&
    (model_df[input_names].notna().all(axis=1))&
    (model_df["from_scrimmage"]==1)&
    (model_df["overtime"]==0)
)


search_rf_play_class_names = ["search_rf_play_" + x for x in search_rf_play_outcome.classes_]
search_rf_drive_class_names = ["search_rf_drive_" + x for x in search_rf_drive_outcome.classes_]
model_df[search_rf_play_class_names] = pd.DataFrame(search_rf_play_outcome.predict_proba(model_df[mask_model_predict][input_names]), index=model_df[mask_model_predict].index)
model_df[search_rf_play_class_names] = model_df[search_rf_play_class_names].fillna(0)
model_df[search_rf_drive_class_names] = pd.DataFrame(search_rf_drive_outcome.predict_proba(model_df[mask_model_predict][input_names]), index=model_df[mask_model_predict].index)
model_df[search_rf_drive_class_names] = model_df[search_rf_drive_class_names].fillna(0)

search_rf_play_class_names_home = [x + "_home" for x in search_rf_play_class_names]
search_rf_play_class_names_away = [x + "_away" for x in search_rf_play_class_names]
search_rf_drive_class_names_home = [x + "_home" for x in search_rf_drive_class_names]
search_rf_drive_class_names_away = [x + "_away" for x in search_rf_drive_class_names]
model_df[search_rf_play_class_names_home] = model_df[search_rf_play_class_names].where(model_df.home_team_has_ball==1, 0)
model_df[search_rf_play_class_names_away] = model_df[search_rf_play_class_names].where(model_df.home_team_has_ball==0, 0)
model_df[search_rf_drive_class_names_home] = model_df[search_rf_drive_class_names].where(model_df.home_team_has_ball==1, 0)
model_df[search_rf_drive_class_names_away] = model_df[search_rf_drive_class_names].where(model_df.home_team_has_ball==0, 0)
display_html(model_df[mask_model][input_names + search_rf_play_class_names + search_rf_drive_class_names].head(15))
# display_html(model_df[mask_model][input_names + search_rf_drive_class_names].head(15))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


,time_left_in_half,half,current_score_diff,current_score_total,cur_spread,cur_over_under,home_timeouts_remaining,away_timeouts_remaining,ytg,yd_from_goal,down,home_team_has_ball,search_rf_play_field_goal_made,search_rf_play_field_goal_missed,search_rf_play_first_down,search_rf_play_none,search_rf_play_offensive_touchdown,search_rf_play_punt,search_rf_play_turnover,search_rf_drive_clock,search_rf_drive_field_goal_made,search_rf_drive_field_goal_missed,search_rf_drive_punt,search_rf_drive_safety,search_rf_drive_touch_down,search_rf_drive_turnover,search_rf_drive_turnover_on_downs
2,1800.0,1.0,0,0,-3.308451,55.707539,3,3,10,75,1,1,0.014819,0.005277,0.235855,0.616493,0.049244,0.059360,0.018953,0.034446,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305
3,1766.0,1.0,0,0,-3.308451,55.707539,3,3,5,70,2,1,0.014819,0.005277,0.235855,0.616493,0.049244,0.059360,0.018953,0.034446,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305
4,1739.0,1.0,0,0,-3.308451,55.707539,3,3,1,66,3,1,0.014819,0.005277,0.235855,0.616493,0.049244,0.059360,0.018953,0.034446,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305
5,1705.0,1.0,0,0,-3.308451,55.707539,3,3,10,55,1,1,0.014819,0.005277,0.235855,0.616493,0.049244,0.059360,0.018953,0.034446,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305
6,1660.0,1.0,0,0,-3.308451,55.707539,3,3,3,48,2,1,0.014819,0.005277,0.235855,0.616493,0.049244,0.059360,0.018953,0.034446,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305
7,1633.0,1.0,0,0,-3.308451,55.707539,3,3,10,44,1,1,0.014819,0.005277,0.235855,0.616493,0.049244,0.059360,0.018953,0.034446,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305
9,1625.0,1.0,0,0,-3.308451,55.707539,3,3,10,81,1,0,0.013595,0.005237,0.216375,0.634968,0.037205,0.071322,0.021299,0.031426,0.123830,0.045365,0.378092,0.001287,0.258144,0.088362,0.073494
10,1584.0,1.0,0,0,-3.308451,55.707539,3,3,9,80,2,0,0.013595,0.005237,0.216375,0.634968,0.037205,0.071322,0.021299,0.031426,0.123830,0.045365,0.378092,0.001287,0.258144,0.088362,0.073494
11,1547.0,1.0,0,0,-3.308451,55.707539,3,3,10,70,1,0,0.013595,0.005237,0.216375,0.634968,0.037205,0.071322,0.021299,0.031426,0.123830,0.045365,0.378092,0.001287,0.258144,0.088362,0.073494
12,1524.0,1.0,0,0,-3.308451,55.707539,3,3,4,64,2,0,0.013595,0.005237,0.216375,0.634968,0.037205,0.071322,0.021299,0.031426,0.123830,0.045365,0.378092,0.001287,0.258144,0.088362,0.073494


### Score Difference Model
* Using the same inputs and adding the outputs of the previous model, we will predict the score differential probablities
* Since this model is an MLP model, we will normalize the inputs. (all features will be made so that the range is from 0 to 1)
* In this model, continuation, null values, end of quarters, and overtime is removed.
* NEW: spread and over/under are replaced by a weighted home and away vegas score prediction. The weight is based on time remaining normalized. So beginning of the game is the full score predictions, half time would be 0.5 * full score predictions

In [24]:
model_df["away_vegas_score_pred"] = np.clip(model_df["cur_over_under"], 30, 80) * 0.5 + model_df["cur_spread"] * 0.5
model_df["home_vegas_score_pred"] = np.clip(model_df["cur_over_under"], 30, 80) * 0.5 - model_df["cur_spread"] * 0.5
model_df["away_vegas_score_pred_weighted"] = model_df["away_vegas_score_pred"] * (model_df["time_left_in_game"] / 3600)
model_df["home_vegas_score_pred_weighted"] = model_df["home_vegas_score_pred"] * (model_df["time_left_in_game"] / 3600)
model_df["point_after_play"] = np.where(model_df["point_after_kick"] + model_df["two_point_attempt"]==1, 1, 0)


# search_mlp_score_diff_clipped_rf_drive_preds = pickle.load(open(os.path.join(root_dir, "models/search_mlp_score_diff_clipped_rf_drive_preds.p"), 'rb'))
# search_mlp_score_diff_clipped_rf_drive_preds = pickle.load(open(os.path.join(root_dir, "models/search_mlp_score_diff_clipped_rf_drive_preds_vegas_adjusted.p"), 'rb'))
model_df["end_of_regulation_score_diff_change_clipped"] = np.clip(model_df["end_of_regulation_score_diff_change"], -50, 50)


# input_names_score_diff_pred = [item for item in input_names if item not in ["punt", "field_goal_attempt"]] + ["kick_off", "point_after_kick", "two_point_attempt"] + ["search_rf_play_first_down_home", "search_rf_play_first_down_away"] + search_rf_drive_class_names_home[1:] + search_rf_drive_class_names_away[1:]

input_names_score_diff_pred = [
    'time_left_in_half',
    'half',
    'current_score_diff',
    'current_score_total',
    # 'home_vegas_score_pred_weighted',
    # 'away_vegas_score_pred_weighted',
    'cur_spread',
    'cur_over_under',
    'home_timeouts_remaining',
    'away_timeouts_remaining',
    'ytg',
    'yd_from_goal',
    'down',
    'home_team_has_ball',
    'kick_off',
    'point_after_play',
    'search_rf_play_first_down_home',
    'search_rf_play_first_down_away',
    'search_rf_drive_field_goal_made_home',
    'search_rf_drive_field_goal_missed_home',
    'search_rf_drive_punt_home',
    'search_rf_drive_safety_home',
    'search_rf_drive_touch_down_home',
    'search_rf_drive_turnover_home',
    'search_rf_drive_turnover_on_downs_home',
    'search_rf_drive_field_goal_made_away',
    'search_rf_drive_field_goal_missed_away',
    'search_rf_drive_punt_away',
    'search_rf_drive_safety_away',
    'search_rf_drive_touch_down_away',
    'search_rf_drive_turnover_away',
    'search_rf_drive_turnover_on_downs_away'

]
output_name = "end_of_regulation_score_diff_change_clipped"

mask_model_score_diff = (
    (model_df.continuation==0)&
    (model_df[input_names_score_diff_pred+[output_name]].notna().all(axis=1))&
    ~(model_df.event_id.isin([12,57,58,13]))&
    (model_df["overtime"]==0)
)
X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val = create_train_test_val_df(model_df[mask_model_score_diff], input_names_score_diff_pred, output_name, normalize=True)
mlp_grid = {
    'hidden_layer_sizes': [(50,50,50), (10,30,10),(100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
score_diff_change_list_clipped = list(model_df.end_of_regulation_score_diff_change_clipped.drop_duplicates().sort_values())

log_loss_scorer_score_diff = make_scorer(log_loss, greater_is_better=False, needs_proba=True, labels=score_diff_change_list_clipped)
mlp_score_diff_search_base = MLPClassifier(verbose=True, early_stopping=True, n_iter_no_change=5, random_state=1)
search_mlp_score_diff_clipped_rf_drive_preds = GridSearchCV(mlp_score_diff_search_base, mlp_grid, cv=cv,n_jobs=-1,verbose=100, scoring=log_loss_scorer_score_diff)
search_mlp_score_diff_clipped_rf_drive_preds.fit(X_train,y_train,groups=group_train)

normalized_score_pred_df = normalize_df(model_df[mask_model_score_diff][input_names_score_diff_pred], model_df[mask_model_score_diff & (model_df.season<2020)][input_names_score_diff_pred])
mlp_search_score_diff_clipped_rf_drive_preds_preds = pd.DataFrame(search_mlp_score_diff_clipped_rf_drive_preds.predict_proba(normalized_score_pred_df.values), index=model_df[mask_model_score_diff].index)
score_diff_clipped_rf_drive_preds_matrix = pd.DataFrame(np.zeros(mlp_search_score_diff_clipped_rf_drive_preds_preds.shape), index=mlp_search_score_diff_clipped_rf_drive_preds_preds.index)
score_diff_change_list_clipped = list(model_df.end_of_regulation_score_diff_change_clipped.drop_duplicates().sort_values())

for column in score_diff_clipped_rf_drive_preds_matrix.columns:
    score_diff_clipped_rf_drive_preds_matrix[column] = score_diff_change_list_clipped[column] + model_df["current_score_diff"]

model_df["xhome_win_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T>0], axis=0)
model_df["xovertime_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T==0], axis=0)
model_df["xaway_win_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T<0], axis=0)
model_df["xend_of_regulation_score_diff_mlp_search_clipped_rf_drive_preds"] = np.sum(score_diff_clipped_rf_drive_preds_matrix * mlp_search_score_diff_clipped_rf_drive_preds_preds, axis=1)

display_html(model_df[["xhome_win_mlp_search_clipped_rf_drive_preds", "xovertime_mlp_search_clipped_rf_drive_preds", "xaway_win_mlp_search_clipped_rf_drive_preds"]].dropna())
display_html(pd.DataFrame(mlp_search_score_diff_clipped_rf_drive_preds_preds.values, columns=score_diff_change_list_clipped))

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3; 2/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=adam
[CV 1/3; 3/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam
[CV 1/3; 1/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam
[CV 3/3; 2/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=adam
[CV 2/3; 2/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=adam
[CV 2/3; 3/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam
[CV 2/3; 1/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam
[CV 3/3; 1/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learnin

/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


Validation score: 0.153556
Validation score: 0.151185


/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/3; 3/24] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam;, score=-3.471 total time=  32.2s
[CV 1/3; 3/24] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam;, score=-3.463 total time=  32.6s
[CV 3/3; 3/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam
[CV 1/3; 4/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=adaptive, solver=adam
Iteration 1, loss = 3.44982853
Iteration 1, loss = 3.45049505
Iteration 1, loss = 3.45101640
Iteration 1, loss = 3.45049505


...

/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


Validation score: 0.156238


/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,xhome_win_mlp_search_clipped_rf_drive_preds,xovertime_mlp_search_clipped_rf_drive_preds,xaway_win_mlp_search_clipped_rf_drive_preds
0,0.705842,0.037698,0.256461
2,0.691698,0.042614,0.265688
3,0.671838,0.043616,0.284546
4,0.647115,0.045379,0.307506
5,0.728288,0.039809,0.231903
...,...,...,...
1930679,0.000275,0.000249,0.999476
1930680,0.000394,0.000333,0.999273
1930683,0.000187,0.000188,0.999626
1930685,0.000118,0.000128,0.999754


,-50,-49,-48,-47,-46,-45,-44,-43,-42,-41,-40,-39,-38,-37,-36,-35,-34,-33,-32,-31,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,1.063882e-03,7.058262e-04,4.602159e-04,2.197077e-04,3.032222e-04,9.690249e-04,6.368054e-04,4.947485e-04,1.112594e-03,9.328005e-04,5.537844e-04,7.176245e-04,1.813745e-03,9.159564e-04,8.641231e-04,2.376468e-03,1.719738e-03,4.361552e-04,2.031661e-03,2.266164e-03,1.840513e-03,1.243668e-03,0.003880,3.724587e-03,0.002112,2.827818e-03,5.114649e-03,2.817427e-03,0.004154,0.007256,0.003870,0.002745,0.007454,0.007497,0.004781,0.003729,0.010564,0.006034,0.005282,0.008388,0.012790,0.004998,0.009207,0.023485,0.011188,0.011246,0.015582,0.026236,0.009679,0.016140,0.037698,0.012260,0.011237,0.037983,0.020246,0.009673,0.016191,0.036687,0.012508,0.007761,0.026927,0.015746,0.008776,0.013995,0.033126,0.012229,0.011999,0.023472,0.017949,0.010155,0.014540,0.026295,0.010803,0.011429,0.023634,1.645302e-02,7.184739e-03,1.440259e-02,0.021005,1.064799e-02,8.053953e-03,1.948269e-02,1.360311e-02,6.496597e-03,1.244845e-02,1.505403e-02,5.872752e-03,6.605758e-03,1.512258e-02,7.688820e-03,3.485357e-03,7.528138e-03,1.485246e-02,4.258290e-03,2.328684e-03,9.152668e-03,4.403837e-03,1.308036e-03,4.014418e-03,1.476870e-02,3.400012e-02
1,9.991768e-04,6.446407e-04,3.946388e-04,1.792092e-04,2.521961e-04,8.870417e-04,5.656977e-04,4.088975e-04,1.048534e-03,8.711686e-04,4.824758e-04,6.531565e-04,1.721502e-03,8.120453e-04,7.434430e-04,2.379482e-03,1.606442e-03,3.920618e-04,1.936264e-03,2.085166e-03,1.694620e-03,1.139210e-03,0.003817,3.533075e-03,0.002094,2.834336e-03,5.137203e-03,2.745533e-03,0.004033,0.007309,0.003590,0.002607,0.007986,0.007599,0.004595,0.003786,0.011317,0.006046,0.005129,0.008598,0.013878,0.004792,0.009313,0.027241,0.011177,0.011478,0.016597,0.028218,0.009833,0.018508,0.042614,0.011888,0.011258,0.043837,0.019687,0.009770,0.016511,0.039144,0.011593,0.007867,0.027107,0.014838,0.007887,0.013778,0.034280,0.011161,0.011812,0.022448,0.016735,0.009979,0.014153,0.025289,0.009729,0.011275,0.023635,1.522029e-02,6.502861e-03,1.348556e-02,0.019777,9.697696e-03,7.555025e-03,1.945055e-02,1.282582e-02,5.632213e-03,1.168834e-02,1.429357e-02,5.103491e-03,5.825464e-03,1.516685e-02,7.039965e-03,2.993501e-03,6.683160e-03,1.418248e-02,3.789771e-03,1.976931e-03,8.520747e-03,3.804895e-03,1.056170e-03,3.524722e-03,1.487115e-02,3.536836e-02
2,1.016872e-03,6.843404e-04,4.128136e-04,1.876062e-04,2.621324e-04,9.207953e-04,5.874099e-04,4.162362e-04,1.123439e-03,9.292925e-04,4.970375e-04,6.884954e-04,1.831759e-03,8.512283e-04,7.611257e-04,2.568687e-03,1.714474e-03,4.217234e-04,2.032630e-03,2.249146e-03,1.782633e-03,1.215898e-03,0.004152,3.733544e-03,0.002201,3.023137e-03,5.567072e-03,2.947321e-03,0.004230,0.007971,0.003907,0.002811,0.008623,0.008354,0.004978,0.004045,0.012336,0.006646,0.005490,0.009482,0.015215,0.005141,0.009898,0.029226,0.011753,0.011857,0.017858,0.029804,0.010117,0.020025,0.043616,0.012427,0.011617,0.045141,0.020392,0.010012,0.016922,0.040021,0.011677,0.008083,0.027338,0.015336,0.007988,0.014051,0.034694,0.011046,0.011746,0.022113,0.016456,0.009709,0.014107,0.024625,0.009503,0.010819,0.022566,1.446337e-02,6.091865e-03,1.281580e-02,0.018904,9.043034e-03,7.114052e-03,1.814632e-02,1.176584e-02,5.091449e-03,1.056156e-02,1.332913e-02,4.577554e-03,5.189509e-03,1.392626e-02,6.443653e-03,2.709555e-03,5.973972e-03,1.281137e-02,3.409769e-03,1.818801e-03,7.526952e-03,3.402263e-03,9.658979e-04,3.144384e-03,1.339085e-02,3.083302e-02
3,1.056408e-03,7.366870e-04,4.368839e-04,1.992735e-04,2.761596e-04,9.643473e-04,6.171883e-04,4.243453e-04,1.232615e-03,1.011255e-03,5.195039e-04,7.400390e-04,1.985825e-03,9.007862e-04,7.877139e-04,2.824191e-03,1.853142e-03,4.603041e-04,2.164403e-03,2.483582e-03,1.909897e-03,1.307840e-03,0.004584,4.011022e-03,0.002332,3

In [22]:
model_df[mask_model_score_diff][input_names_score_diff_pred+ ["continuation", output_name]]

,time_left_in_half,half,current_score_diff,current_score_total,cur_spread,cur_over_under,home_timeouts_remaining,away_timeouts_remaining,ytg,yd_from_goal,down,home_team_has_ball,kick_off,point_after_play,search_rf_play_first_down_home,search_rf_play_first_down_away,search_rf_drive_field_goal_made_home,search_rf_drive_field_goal_missed_home,search_rf_drive_punt_home,search_rf_drive_safety_home,search_rf_drive_touch_down_home,search_rf_drive_turnover_home,search_rf_drive_turnover_on_downs_home,search_rf_drive_field_goal_made_away,search_rf_drive_field_goal_missed_away,search_rf_drive_punt_away,search_rf_drive_safety_away,search_rf_drive_touch_down_away,search_rf_drive_turnover_away,search_rf_drive_turnover_on_downs_away,continuation,end_of_regulation_score_diff_change_clipped
0,1800.0,1.0,0,0,-3.308451,55.707539,3,3,-1,-1,0,1,1,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,41
2,1800.0,1.0,0,0,-3.308451,55.707539,3,3,10,75,1,1,0,0,0.235855,0.0,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,41
3,1766.0,1.0,0,0,-3.308451,55.707539,3,3,5,70,2,1,0,0,0.235855,0.0,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,41
4,1739.0,1.0,0,0,-3.308451,55.707539,3,3,1,66,3,1,0,0,0.235855,0.0,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,41
5,1705.0,1.0,0,0,-3.308451,55.707539,3,3,10,55,1,1,0,0,0.235855,0.0,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930679,43.0,2.0,-15,51,3.000000,53.000000,2,2,10,57,1,1,0,0,0.235855,0.0,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1930680,39.0,2.0,-15,51,3.000000,53.000000,2,2,10,46,1,1,0,0,0.235855,0.0,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1930683,34.0,2.0,-15,51,3.000000,53.000000,1,2,22,58,2,1,0,0,0.235855,0.0,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1930685,28.0,2.0,-15,51,3.000000,53.000000,0,2,28,64,3,1,0,0,0.235855,0.0,0.133544,0.043166,0.315783,0.000739,0.328911,0.081106,0.062305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [27]:
model_df

,game_code,game_date,season,home_team_id,home_team,home_team_abbrev,home_conf_name,home_division_type,away_team_id,away_team,away_team_abbrev,away_conf_name,away_division_type,home_final_score,away_final_score,final_score_diff,end_of_regulation_score_diff,home_rest_of_game_score,away_rest_of_game_score,end_of_regulation_score_diff_change,home_score_added,away_score_added,current_score_diff,current_score_total,home_start_score,away_start_score,home_team_outcome,home_team_win,draw,away_team_win,nevent,quarter,overtime,home_team_has_ball,off_team_id,def_team_id,kick_off,punt,point_after_kick,two_point_attempt,field_goal_attempt,off_start_score,off_end_score,off_score_change,def_start_score,def_end_score,def_score_change,play_counts,efficiency_counts,from_scrimmage,first_down,scoring_play,possession_change,continuation,event_name,event_id,yards_gained,drive_outcome_id,drive_outcome_desc,down,ytg,yd_from_goal,drive_id,drive_start,play_start_time,cur_spread,cur_over_under,half,home_timeout,away_timeout,home_timeouts_remaining,away_timeouts_remaining,time_left_in_game,sequence,play_start_id,turnover,touchdown_scored,fieldgoal_made,touchdown_in_play,turnover_in_play,field_goal_in_play,first_down_in_play,play_outcome,drive_outcome_desc_basic,end_of_regulation_score_total_diff,time_left_in_half,search_rf_play_field_goal_made,search_rf_play_field_goal_missed,search_rf_play_first_down,search_rf_play_none,search_rf_play_offensive_touchdown,search_rf_play_punt,search_rf_play_turnover,search_rf_drive_clock,search_rf_drive_field_goal_made,search_rf_drive_field_goal_missed,search_rf_drive_punt,search_rf_drive_safety,search_rf_drive_touch_down,search_rf_drive_turnover,search_rf_drive_turnover_on_downs,search_rf_play_field_goal_made_home,search_rf_play_field_goal_missed_home,search_rf_play_first_down_home,search_rf_play_none_home,search_rf_play_offensive_touchdown_home,search_rf_play_punt_home,search_rf_play_turnover_home,search_rf_play_field_goal_made_away,search_rf_play_field_goal_missed_away,search_rf_play_first_down_away,search_rf_play_none_away,search_rf_play_offensive_touchdown_away,search_rf_play_punt_away,search_rf_play_turnover_away,search_rf_drive_clock_home,search_rf_drive_field_goal_made_home,search_rf_drive_field_goal_missed_home,search_rf_drive_punt_home,search_rf_drive_safety_home,search_rf_drive_touch_down_home,search_rf_drive_turnover_home,search_rf_drive_turnover_on_downs_home,search_rf_drive_clock_away,search_rf_drive_field_goal_made_away,search_rf_drive_field_goal_missed_away,search_rf_drive_punt_away,search_rf_drive_safety_away,search_rf_drive_touch_down_away,search_rf_drive_turnover_away,search_rf_drive_turnover_on_downs_away,away_vegas_score_pred,home_vegas_score_pred,away_vegas_score_pred_weighted,home_vegas_score_pred_weighted,point_after_play,end_of_regulation_score_diff_change_clipped,xhome_win_mlp_search_clipped_rf_drive_preds,xovertime_mlp_search_clipped_rf_drive_preds,xaway_win_mlp_search_clipped_rf_drive_preds,xend_of_regulation_score_diff_mlp_search_clipped_rf_drive_preds,end_of_regulation_score_total_diff_clipped
0,1178084,2012-08-30 17:15:00,2012,3499,Utah Utes,Utah,Pac-12,FBS,3710,Northern Colorado Bears,NorCol,Big Sky,FCS,41,0,41,41,41,0,41,0,0,0,0,0,0,W,1,0,0,1,1,0,1,3710,3499,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,Kick Off,5,65.0,NaN,None,0,-1,-1,NaN,3600.0,900.0,-3.308451,55.707539,1.0,0,0,3,3,3600.0,0,1,0,0,0,0,0,0,0,none,NaN,41,1800.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.199544,29.507995,26.199544,29.507995,0,41,0.705842,0.037698,0.256461,12.244753,41
1,1178084,2012-08-30 17:15:00,2012,3499,Utah Utes,Utah,Pac-12,FBS,3710,Northern Colorado Bears,NorCol,Big Sky,FCS,41,0,41,41,41,0,41,0,0,0,0,0,0,W,1,0,0,2,1,0,1,3499,3710,

In [30]:
model_df["end_of_regulation_score_total_diff_clipped"] = np.clip(model_df["end_of_regulation_score_total_diff"], 0, 105)


output_name = "end_of_regulation_score_total_diff_clipped"
input_names_total_score_pred = list(search_mlp_score_diff_clipped_rf_drive_preds.feature_names_in_) + ["xend_of_regulation_score_diff_mlp_search_clipped_rf_drive_preds"]
X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val = create_train_test_val_df(model_df[mask_model_score_diff], input_names_total_score_pred, output_name, normalize=True)


mlp_total_score_search_base = MLPClassifier(verbose=True, early_stopping=True, n_iter_no_change=5, random_state=1)
search_mlp_total_score = GridSearchCV(mlp_total_score_search_base, mlp_grid, cv=cv,n_jobs=-1,verbose=100, scoring=log_loss_scorer)
search_mlp_total_score.fit(X_train,y_train,groups=group_train)


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3; 2/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=adam
[CV 1/3; 1/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam
[CV 2/3; 1/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam
[CV 2/3; 2/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=adam
[CV 3/3; 1/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam
[CV 3/3; 2/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=adam
[CV 1/3; 3/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam
[CV 2/3; 3/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learnin

/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.

[CV 2/3; 3/24] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam;, score=-3.344 total time=  22.9s
[CV 1/3; 3/24] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam;, score=-3.337 total time=  23.1s


/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-

[CV 1/3; 4/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=adaptive, solver=adam
[CV 3/3; 3/24] START activation=tanh, alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam


...

[CV 2/3; 6/24] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=adam;, score=-3.340 total time=  27.1s
[CV 3/3; 6/24] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=adam;, score=-3.337 total time=  27.5s


/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


[CV 1/3; 9/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam
[CV 2/3; 9/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(10, 30, 10), learning_rate=constant, solver=adam


/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


KeyboardInterrupt: 